# CLIP

![clip](../images/CLIP_tutorial/CLIP.png)

## 1 模型

### 1.1 结构

![clip_pretrain](../images/CLIP_tutorial/clip_pretrain.jpg)

#### 1.1.1  
问：什么是CLIP？  
答：CLIP的英文全称是Contrastive Language-Image Pre-training，即一种基于对比文本-图像对的预训练方法（模型）。

#### 1.1.2  
问：[什么是对比学习？](https://ankeshanand.com/blog/2020/01/26/contrative-self-supervised-learning.html)    
注：问题链接是一片讲解对比学习的知名博文，以下关于对比学习的回答由此展开。  
答：  
自监督学习（一种机器学习范式，模型自行从无标注数据学习）当前被分为两类：  
（1）生成式方法（Generative Methods）  
（2）对比式方法（Contrastive Methods）  
生成式方法（Generative Methods）这类方法以自编码器为代表，主要关注像素级损失。扩散模型是典型的生成式自监督模型（回忆DDPM原理，预测噪声的MSE损失函数是不是“像素级”的）。  
生成式方法的缺点：  
（1）使用像素级的损失可能导致模型更关心像素级的细节而忽略（图片中）更抽象的潜在因子。  
（2）基于像素的目标通常假设像素间的独立性，从而降低了它们对相关性或复杂结构建模的能力。  
![one dollar](../images/CLIP_tutorial/one_dollar.png)  
上图左边是让人根据回忆画出的一美元钞票，右边是对着原版一美元钞票画出来的。  
这种现象引发人们的思考：我们见过无数次一美元钞票（对美国人来说），但是却没办法保留它的完整特征，我们只是保留足够特征让它和其他物品区分开。尽管右边的钞票细节更丰富，但左边的钞票丝毫不影响人们把它认出来。这是一个很有趣的现象，这是否意味着我们可以构建不专注于像素细节，而只编码足以区分不同对象的高级特征表示的学习算法？  
答案是肯定的：  
对比式方法（Contrastive Methods）是通过将数据分别与正例样本和负例样本在特征空间进行对比，来学习样本的特征表示。因此对比学习不会过分关注像素细节，而能够关注抽象的语义信息，并且相比于像素级别的重构，优化也变得更加简单。对比学习主要的难点在于如何构造正负样本。

<a id='section_1_1_3'></a>
#### 1.1.3  
问：对比式方法是如何工作的？  
答：  
对比学习的目标是学习一个编码器（模型），使得这个编码器对正样本的评分远远大于负样本（正负样本差距尽可能大）：  
![contrastive method](../images/CLIP_tutorial/contrastive_method.jpg)  
上图中的x一般称为锚点（Anchor），锚点是用来和正负样本比较的点（注意这个概念，锚点既不是真实值也不是预测值，拿什么数据点比什么点才是锚点，此处易混淆）。对比学习中，我们希望锚点接近正样本（分数高），远离负样本（分数低）。    
我们先看博客中给出的对比学习的损失函数infoNCE loss：  
![contrastive loss](../images/CLIP_tutorial/contrastive_loss.jpg)  
这个公式是不是有些熟悉？我们把公式拆解来看，log内部的公式是不是有些眼熟：  
![softmax](../images/CLIP_tutorial/softmax.png)  
softmax函数大家应该非常熟悉，这个函数一般用来把神经网络输出变成一个概率表示。但是在对比学习中称其为概率是不妥的：我们infoNCE loss最内层分子的目的是最大化锚点和正样本相似性，分母中的项考虑了正样本和所有负样本的相似性，这个形式实际上是一个softmax形式的“似然”。输出分数的计算方式使用的是点积（余弦相似度）：  
![contrastive score function](../images/CLIP_tutorial/contrastive_score_function.jpg)  
再看infoNCE外部，先不看期望符号，这个负对数似然的形式是不是很像图像分类中的交叉熵损失函数：  
![cross entropy](../images/CLIP_tutorial/cross_entropy.png)  
其中p是真实概率分布，q是模型预测的概率分布。  
实际上，这就是一个交叉熵损失函数。可能有同学有疑惑，这里只有q并没有真实概率分布p怎么是交叉熵呢？  
再回忆对比学习对正负样本的定义，对于一个锚点x，它真实的概率分布是类似one-hot（独热编码）的，正样本为1，负样本为0，因此公式中其实是隐藏了正样本乘以一个为1的真实概率分布，而负样本为0全部不计算。举个例子，假定在对比学习时有1个正样本、2个负样本，它们的独热编码是1、0、0，交叉熵计算方式如下：  
![cross entropy ex](../images/CLIP_tutorial/cross_entropy_ex.png)  
现在我们考虑完整的infoNCE loss，稍微和上面博文中的公式有所不同，对于一个正样本：  
![infoNCE single](../images/CLIP_tutorial/infoNCE_single.png)  
这里多加了一个温度系数τ，这个系数可以是可学习的，也可以设置成超参数人为指定。我们可以先不深究温度系数的作用，简单认为τ的位置乘了一个缩放系数即可。  
同时我们注意到，整体的损失函数肯定不能是一个正样本的损失，应该设置成所有正样本损失的均值：  
![avg](../images/CLIP_tutorial/avg.png)   
这个就是期望的定义，把上面博文公式中没有解释的期望加进去得到完整的infoNCE loss：  
![infoNCE all](../images/CLIP_tutorial/infoNCE_all.png)  
有了损失函数的情况下，只需要定义出正、负样本，即可开始训练对比学习模型。

#### 1.1.4  
问：对比学习和分类问题都用到了交叉熵损失函数，可以说对比学习的本质是分类吗？   
答：
不可以，对比学习和分类问题求解上有着一些相似之处，但它们的目标完全是不同的。  
对比学习的目标是通过“对比”，学习一个空间表示，其中相似的样本被拉近，而不同的样本被推远。在这个空间中，相似性是根据某种度量（如欧几里得距离或余弦相似度）来定义的。而分类问题要学习的是每个类别的概率表示，学习的是一个具体的类别标签。
有趣的是，根据1.1.3中的分析，当锚点、正样本、负样本给定的条件下，我们可以将其当成一个多分类问题来计算，这也是对比学习和分类问题看起来有点相似的原因。

#### 1.1.5  
问：对比学习中，每个锚点只能有一个正样本吗？  
答：不是的，对比学习中每个锚点可以有一个或多个正样本，这取决于算法的设置。一个正样本可以简化学习过程、减少计算量。我们学习的CLIP每个锚点恰好对应一个正样本。

#### 1.1.6  
问：softmax算的不是概率吗？为什么在[1.1.3](#section_1_1_3)中说是求似然？  
答：  
概率和似然的概念非常相似，但是目的不同。  
概率描述了已知参数下观察到某一数据的可能性。似然描述了在给定观察到的数据下，不同参数的可能性。  
回到[1.1.3](#section_1_1_3)，我们实际上是计算了正样本与锚点相对于所有其他负样本的相似性的"可能性"或"似然"。当此值越大时，正样本与锚点的相似度越高，而与负样本的相似度越低。因此，尽管softmax给出的是概率，但在这种特定的情境下，我们可以将其解释为似然。这种解释更多是概念上的，而不是严格数学意义上。对比学习中的这种"似然"实际上是建立在softmax形式上的，但其目的是量化某种相似性或区分度，这与传统统计中的似然定义略有不同。  
对比学习中应用它的主要原因是softmax具有的“马太效应”：  
观察一个序列1、2、3，它的softmax输出是0.09、0.24、0.67；如果我们把输入序列调大一点，比如1、2、5，它的softmax输出变成0.02、0.05、0.93。可见softmax会放大大的输出，缩小小的输出。这一点在多分类问题中时常被诟病，比如模型预测图片类别，输出马的分数是0.8、驴的分数是0.6，由于马和驴有很高的相似度，这个分数似乎是合理的。但如果加softmax输出结果，马和驴的分数差距就会被softmax放大，这时人类观察模型输出可能会认为这是两个相差很多的类别（实际并不是）。当然在对比学习中没有这个困扰，我们希望正负样本差距越大越好，softmax这种性质非常合适。

<a id='section_1_1_7'></a>
#### 1.1.7
问：对比学习中，温度系数的作用是什么？  
答：  
对比学习损失函数（infoNCE loss）是一个具备困难负样本自发现性质的损失函数，温度系数的作用是调节对困难样本的关注程度：越小的温度系数越关注于将正样本和相似的负样本分开（加大对负样本惩罚），意味着对困难样本有更好的区分。但这并不意味越小的温度系数越好，因为很小的温度系数对错误数据（比如把正样本分到负样本去）容忍度低（惩罚高），这会导致模型学习到错误的信息，而大一点的温度系数对错误数据容忍度高，对数据鲁棒性更强。具体请看论文[Understanding the Behaviour of Contrastive Loss](https://arxiv.org/pdf/2012.09740.pdf)。  

#### 1.1.8  
问：为什么对比学习使用infoNCE如此复杂的损失函数，而不是简单的把模型的输出乘以系数求和（或其他各种简单形式的损失函数）？  
答：参考[1.1.7](#section_1_1_7)，无论是softmax还是交叉熵其实都在做一件事：拉大正负样本间的差距。简单的损失函数做不到这一点，不具备困难负样本自发现的性质。

#### 1.1.9  
问：CLIP是如何定义正负样本的？  
答：一个minibatch中n个文本-图片对，和自己匹配的一对是正样本，其他全部是负样本。因此正样本只出现在对角线上。有了正负样本定义和对比学习损失函数的理解，CLIP理论上已经可以训练。

#### 1.1.10  
问：CLIP是用什么数据集训练的？  
答：openai收集互联网上4亿个文本-图片对数据进行训练，可以预见其中包含大量数据清洗的工作，遗憾的是数据清洗步骤、数据集和详细的数据来源openai并没有提供。

### 1.2 伪代码

![clip_numpylike_pseudocode](../images/CLIP_tutorial/clip_numpylike_pseudocode.jpg)  

#### 1.2.1
问：伪代码中的参数如何理解？  
答：  
image encoder： 图像编码器。作用是提取图像特征，可以使用任意提取图像特征的模型（论文中的例子给的resnet和视觉transformer）。  
text encoder： 文本编码器，扩散模型中使用的文本编码器正是这部分。作用是提取文本特征，同样可以使用任意提取文本特征的模型（论文中的例子给的resnet和文本transformer）。  
I：输入的图像，需要大小对齐，维度是nhwc。n：minibatch的数量，也就是我们训练时取的一批次图片的数量；h：图片高；w：图片宽；c：图片通道数。  
T：输入的文本，需要大小对齐，维度是nl。n：n：minibatch的数量，同I中n是相同的，因为文本-图片对一一应对。l：文本的长度。  
I_f：图像编码器输出，每张图片输出一个特征向量，维度d_i，共n张图，整体维度n * d_i。  
T_f：文本编码器输出，每个文本输出一个特征向量，维度d_t，共n个文本，整体维度n * d_t。  
W_i：图片特征向量嵌入（embedding）矩阵，全部参数可学习。意图是把图像的特征向量和文本的特征向量投影到相同维度以进行多模态融合。形状 d_i * d_e，d_e是文本-图片特征统一投影的维度。  
W_t：文本特征向量嵌入（embedding）矩阵，全部参数可学习。意图是把图像的特征向量和文本的特征向量投影到相同维度以进行多模态融合。形状 d_t * d_e，d_e是文本-图片特征统一投影的维度。  
I_e：图片经过嵌入后投影的新维度特征，形状 n * d_e，和文本特征对齐。  
T_e：文本经过嵌入后投影的新维度特征，形状 n * d_e，和图片特征对齐。  
t：可学习的温度，一个标量。  
logits：深度学习中的常用术语，表示模型的原始输出。CLIP中维度是 n * n，参考模型结构图，这个 n * n 的矩阵即是图中最后得到的矩阵。    
labels：文本-图片对真值，也就是CLIP定义的正样本。n * n 矩阵有n个正样本。  
loss_i：图片分支的损失值。  
loss_t：文本分支的损失值。  
loss：整体损失值。

#### 1.2.2  
问：logits是怎么算的余弦相似度？  
答：  
余弦相似度计算公式：  
![cos_sim](../images/CLIP_tutorial/cosine_sim.png)  
公式分子是点积，分母是模长乘积，模长在深度学习中一般用L2范数表示。假设有向量v，它的L2范数计算方法：  
![L2](../images/CLIP_tutorial/L2.png)  
logits上一步的l2_normalize表示L2归一化，即用L2范数归一化向量，归一化后的向量计算方法如下：  
![L2_norm](../images/CLIP_tutorial/L2_norm.png)  
可见两步结合，点乘结束得到的即为余弦相似度（归一化在前一步做的）。求出余弦相似度后，又乘一个可学习的温度系数得到模型输出logits。

#### 1.2.3  
问：为什么要进行归一化，可不可以不做归一化？  
答：一般来说，归一化操作可以让数据获得更好的分布，从而加快训练速度。不做归一化直接点积相乘当然也是可以的，归一化操作除以的是一个标量，去掉之后如果不考虑数据分布对训练不产生影响。但是为了更好的分布（更快的训练速度）还是做归一化。

#### 1.2.4  
问：labels为什么能产生正样本的标签？图片和文本的正样本标签为什么是相同的？  
答：观察labels的计算方法，np.arange(n)产生的是0、1、2...n的数组。再观察模型结构图中 n * n 的输出，正样本全部在对角线上。因此不管是对于横轴（axis=0，表示图片分支）还是对于纵轴（axis=1，表示文本分支），它们的正样本标签都是一样的，即第i个嵌入的正样本位置是i，其余均是负样本。

#### 1.2.5  
问：为什么使用交叉熵（cross entropy）作为损失函数？  
答：参考[1.1.3](#section_1_1_3)

#### 1.2.6  
问：为什么没有infoNCE loss中的softmax步骤？  
答：cross_entropy_loss中包含softmax，现代深度学习框架（如pytorch）在计算cross entropy loss的时候通常默认计算softmax，而不需要显式声明。大家在自己写代码时也需要注意这一点，如果重复进行softmax操作可能导致模型难以收敛或不收敛。

## 2 zero-shot分类

![clip_cls_zero_shot](../images/CLIP_tutorial/clip_cls_zero_shot.jpg)

### 2.1 介绍

CLIP有不弱的zero-shot（零样本学习，即无需训练可直接工作）分类的能力，也是论文的一大亮点，其中用了一些小技巧会在本节说明。但需要注意的是，零样本分类完全不是CLIP最吸引人的地方，这点随着时间的推移基本已有共识：CLIP的最大贡献无疑是把自然语言级别的抽象概念引入计算机视觉（开山之作），通过文本-图像关系的建立为计算机视觉开辟一片新天地。大家自学时不要被CLIP大篇幅的zero-shot实验带偏关注方向。

### 2.2 问答

#### 2.2.1  
问：图片分类问题中，每个类别对应一个词或词组。而CLIP的输入是文本-图片对，zero-shot分类是如何处理类别进行推理的？  
答：这里应用自然语言处理中提示模板（prompt template）的技巧，把不同类别词汇嵌入预先设置好的提示模板，形成一句话，n个类别就有n句话，n句话对应n个文本特征嵌入（embedding）。图片通过图片编码器得到一个特征嵌入（embedding），用一个图片特征嵌入去和n个文本嵌入做点积，得到的n个值中的最大值对应的类别就是图片的分类结果。

#### 2.2.2  
问：为什么要用提示模板把类别词汇变成一个句子，直接输入类别词汇到文本编码器不行吗？  
答：当然可以，不过在深度学习中，推理的处理需要和训练的处理保持一致。CLIP训练时使用的文本-图像对中的文本都是句子，推理时如果换成单词肯定会影响效果。

#### 2.2.3  
问：CLIP分类的优势是什么？  
答：  
CLIP分类最大的优势无疑是继承自对比学习对抽象概念的理解优势。我们会发现相比传统模型（resnet等），无论是视觉语义还是文本语义，CLIP都能理解极为抽象的概念，这意味着它能学到更加高级的“概念”语义，这是传统模型学习像素级别的语义无法理解的。CLIP官网中有一张香蕉图，很好的说明CLIP对“香蕉”概念的理解：  
![clip banana](../images/CLIP_tutorial/clip_banana.png)

## 3 相关资料
论文： [Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/pdf/2103.00020.pdf)  
[openai官方仓库](https://github.com/openai/CLIP)：仅开源部分代码，有模型和权重，能用来做推理，但是并不提供训练代码。  
[openai CLIP官方页面](https://openai.com/research/clip)  
[b站讲解](https://www.bilibili.com/video/BV1SL4y1s7LQ/?buvid=4672c179b3e1ee9116c2b795f3d3061a&is_story_h5=false&mid=8JJ%2Fot%2BaDbPrUImwdIrMww%3D%3D&p=1&plat_id=116&share_from=ugc&share_medium=iphone&share_plat=ios&share_session_id=8CD31657-D253-413A-9758-A395C26332EE&share_source=WEIXIN&share_tag=s_i&timestamp=1693463757&unique_k=rIhhRL2&up_id=1567748478)  
[open clip](https://github.com/mlfoundations/open_clip)： 一个CLIP开源框架，根据CLIP原理复现的完整功能的第三方代码库。可以训练自己的CLIP模型，同时提供比官方模型效果更好的第三方预训练模型。